# Análisis de Iteraciones  
Este cuaderno facilita el flujo de evaluación de rendimiento de las distintas iteraciones del modelo. Su función principal es centralizar las métricas (RMSE y $R^2$) extraídas de los metadatos de cada experimento para realizar un análisis comparativo. Mediante la generación de rankings y visualizaciones interactivas de complejidad frente a error, el _script_ facilita la selección de la configuración óptima basada en el compromiso entre el número de variables utilizadas y la precisión obtenida.

## 1. Importación de librerías e inicialización

In [1]:
import os
import glob
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Config
ITERATIONS_DIR = "" 
FILE_PATTERN = "*_meta.json"

## 2. Carga de iteraciones

In [2]:
def load_experiments(folder):
    files = glob.glob(os.path.join(folder, FILE_PATTERN))
    print(f"Encontrados {len(files)} experimentos en '{folder}'...")
    
    data = []
    
    for file_path in files:
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
                
            cfg = meta.get("cfg", {})
            metrics = meta.get("metrics_global", {})
            by_day = meta.get("metrics_by_day", [])
            
            # Basic info
            row = {
                "id": cfg.get("iteration_id", "unknown"),
                "short_id": meta.get("exp_id", "unknown")[:15], 
                "description": cfg.get("description", "").strip(),
                "model_depth": cfg.get("model_params", {}).get("max_depth", "?"),
                "n_features": len(meta.get("features", [])),
                "features_list": ", ".join(meta.get("features", [])),
                
                # Global metrics
                "RMSE_Global": metrics.get("rmse", 9999),
                "R2_Global": metrics.get("r2", 0),
            }
            
            # Metrics by day 
            for day_stat in by_day:
                date = day_stat["date"]
                row[f"RMSE_{date}"] = day_stat["rmse"]
                row[f"R2_{date}"] = day_stat["r2"]
                
            data.append(row)
            
        except Exception as e:
            print(f"Error leyendo {file_path}: {e}")

    return pd.DataFrame(data)

## 3. Análisis y visualización

In [3]:
def analyze_and_plot(df):
    if df.empty:
        print("No hay datos para analizar.")
        return

    # Ordering by best global RMSE
    df = df.sort_values("RMSE_Global", ascending=True)
    
    print("\n" + "="*50)
    print("  MEJORES MODELOS")
    print("="*50)
    
    best_global = df.iloc[0]
    print(f"\n🌍 MEJOR GLOBAL: {best_global['id']}")
    print(f"   RMSE: {best_global['RMSE_Global']:.2f} | R2: {best_global['R2_Global']:.3f}")
    print(f"   Features ({best_global['n_features']}): {best_global['features_list']}")
    
    day_cols = [c for c in df.columns if c.startswith("RMSE_20")]
    for col in day_cols:
        date = col.replace("RMSE_", "")
        best_day = df.sort_values(col, ascending=True).iloc[0]
        print(f"\n📅 MEJOR DÍA {date}: {best_day['id']}")
        print(f"   RMSE: {best_day[col]:.2f}")

    # Pandas DataFrame - Tabla Comparativa
    print("\n" + "="*50)
    print("📊 TABLA COMPARATIVA")
    print("="*50)
    cols_show = ["id", "n_features", "model_depth", "RMSE_Global"] + day_cols
    print(df[cols_show].head(10).to_string(index=False))

    #  Plotly Scatter Plot
    df["hover_text"] = (
        "<b>" + df["id"] + "</b><br>" +
        "<i>" + df["description"] + "</i><br><br>" +
        "Features: " + df["features_list"]
    )

    fig = px.scatter(
        df,
        x="n_features",
        y="RMSE_Global",
        color="model_depth",  
        size="R2_Global",     
        hover_name="id",
        hover_data={
            "id": False, 
            "description": True, 
            "n_features": False,
            "RMSE_Global": ":.2f"
        },
        text="id",            
        title="Comparativa de Experimentos: Complejidad vs Error",
        labels={"n_features": "Número de Variables (Features)", "RMSE_Global": "Error Global (RMSE)"},
        template="plotly_dark"
    )

    fig.update_traces(textposition='top center')
    fig.update_layout(height=600)
    
    
    fig.add_hline(y=df["RMSE_Global"].min(), line_dash="dash", line_color="green", annotation_text="Mejor Global")

    
    fig.show()

if __name__ == "__main__":
    df = load_experiments(ITERATIONS_DIR)
    analyze_and_plot(df)

Encontrados 10 experimentos en ''...

  MEJORES MODELOS

🌍 MEJOR GLOBAL: it_025_cloud_optimization
   RMSE: 124.64 | R2: 0.831
   Features (7): sim_irradiance_wm2, cloud_opacity, cloud_diff, cloud_std_60m, cloud_mean_60m, day_volatility, day_cloudiness

📅 MEJOR DÍA 2025-04-11: it_003_meteo
   RMSE: 110.12

📅 MEJOR DÍA 2025-04-20: it_025_cloud_optimization
   RMSE: 181.73

📅 MEJOR DÍA 2025-10-07: it_028_minimal_optimization
   RMSE: 27.82

📊 TABLA COMPARATIVA
                         id  n_features  model_depth  RMSE_Global  RMSE_2025-04-11  RMSE_2025-04-20  RMSE_2025-10-07
  it_025_cloud_optimization           7            6   124.643303       110.894829       181.730401        35.836531
it_032_minimal_optimization           2            5   125.521165       111.601123       184.054910        30.588689
             it_002_minimal           2            5   125.710189       111.821418       184.221937        31.103111
               it_015_cloud           7            4   128.261049    